In [1]:
# Read from Cosmos DB analytical store into a Spark DataFrame and display 10 rows from the DataFrame
# To select a preferred list of regions in a multi-region Cosmos DB account, add .option("spark.cosmos.preferredRegions", "<Region1>,<Region2>")

df = spark.read\
    .format("cosmos.olap")\
    .option("spark.synapse.linkedService", "asacosmosdbnm10djs")\
    .option("spark.cosmos.container", "OnlineUserProfile01")\
    .load()

display(df.limit(10))

StatementMeta(SparkPool01, 7, 1, Finished, Available)

SynapseWidget(Synapse.DataFrame, adf04c6d-9f12-4423-a750-a2ffa96799b7)

In [2]:
unwanted_cols = {'_attachments','_etag','_rid','_self','_ts','collectionType','id'}

# Remove unwanted columns from the columns collection
cols = list(set(df.columns) - unwanted_cols)

profiles = df.select(cols)

display(profiles.limit(10))

StatementMeta(SparkPool01, 7, 2, Finished, Available)

SynapseWidget(Synapse.DataFrame, 1e397ebb-cda7-4915-8e8c-21256483f451)

In [4]:
profiles.count()

StatementMeta(SparkPool01, 7, 4, Finished, Available)

99999

In [5]:
from pyspark.sql.functions import udf, explode

preferredProductsFlat=profiles.select('userId',explode('preferredProducts').alias('productId'))
productReviewsFlat=profiles.select('userId',explode('productReviews').alias('productReviews'))
display(productReviewsFlat.limit(10))

StatementMeta(SparkPool01, 7, 5, Finished, Available)

SynapseWidget(Synapse.DataFrame, b1653003-f3cf-40ed-b781-e5383d458eef)

In [6]:
display(preferredProductsFlat.limit(20))

StatementMeta(SparkPool01, 7, 6, Finished, Available)

SynapseWidget(Synapse.DataFrame, f1e08e99-89ab-4c36-a8b1-cd3002692820)

In [7]:
productReviews = (productReviewsFlat.select('userId','productReviews.productId','productReviews.reviewText')
    .orderBy('userId'))

display(productReviews.limit(10))

StatementMeta(SparkPool01, 7, 7, Finished, Available)

SynapseWidget(Synapse.DataFrame, f75ba180-fb88-4abd-9a8a-deb28770eed4)

In [9]:
preferredProductReviews = (preferredProductsFlat.join(productReviews,
    (preferredProductsFlat.userId == productReviews.userId) &
    (preferredProductsFlat.productId == productReviews.productId))
)

display(preferredProductReviews.limit(100))

StatementMeta(SparkPool01, 7, 9, Finished, Available)

SynapseWidget(Synapse.DataFrame, c502ad5c-4608-4e25-acc6-86a91da670f1)